In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from sklearn.preprocessing import StandardScaler

In [2]:
dataset = pd.read_csv("data.csv")

In [3]:
#df.head()
#df["vol"] = df["vol"].str.replace(',', '').astype(float)
pd.read_csv('data.csv').shape

(1910, 6)

In [4]:
import talib

In [5]:
dataset = dataset.dropna()
dataset = dataset[['Open', 'High', 'Low', 'Close']]
    

In [6]:
dataset['H-L'] = dataset['High'] - dataset['Low']
dataset['O-C'] = dataset['Close'] - dataset['Open']
    
dataset['ma_5'] = dataset['Close'].rolling(window=5).mean()
dataset['ma_10'] = dataset['Close'].rolling(window=10).mean()
    
dataset['EWMA_12'] = dataset['Close'].ewm(span=12).mean()

dataset['std_5'] = dataset['Close'].rolling(window=5).std()
dataset['std_10'] = dataset['Close'].rolling(window=10).std()
    

In [7]:
dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod = 14)
dataset['Williams %R'] = talib.WILLR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)
dataset['SAR'] = talib.SAR(np.array(dataset['High']), np.array(dataset['Low']), 0.2, 0.2)
dataset['ADX'] = talib.ADX(np.array(dataset['High']), np.array(dataset['Low']), np.array(dataset['Close']), 10)


In [8]:
dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)


In [9]:
dataset.head()

,Open,High,Low,Close,H-L,O-C,ma_5,ma_10,EWMA_12,std_5,std_10,RSI,Williams %R,SAR,ADX,Price_Rise
0,873.0,879.0,851.0,879.0,28.0,6.0,NaN,NaN,879.000000,NaN,NaN,NaN,NaN,NaN,NaN,1
1,879.0,913.0,882.0,900.0,31.0,21.0,NaN,NaN,890.375000,NaN,NaN,NaN,NaN,851.0,NaN,0
2,900.0,916.0,892.0,892.0,24.0,-8.0,NaN,NaN,891.009238,NaN,NaN,NaN,NaN,863.4,NaN,0
3,892.0,875.0,842.0,865.0,33.0,-27.0,NaN,NaN,882.799138,NaN,NaN,NaN,NaN,916.0,NaN,0
4,865.0,875.0,858.0,858.0,17.0,-7.0,878.8,NaN,876.061301,17.655028,NaN,NaN,NaN,916.0,NaN,0


In [10]:
#dataset.hist(bins = 50, figsize =(20,15))   #shows histogram
#plt.show()

In [11]:
from sklearn import svm
from collections import Counter


In [12]:
dataset = dataset.dropna()
X = dataset.iloc[:, 4:-1]
y = dataset.iloc[:, -1]

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [14]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=8, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [15]:
y_pred = svclassifier.predict(X_test)

In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[203   1]
 [173   2]]
              precision    recall  f1-score   support

           0       0.54      1.00      0.70       204
           1       0.67      0.01      0.02       175

    accuracy                           0.54       379
   macro avg       0.60      0.50      0.36       379
weighted avg       0.60      0.54      0.39       379

